<a href="https://colab.research.google.com/github/ZubinO/independentStudy/blob/main/ScrapingReddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install praw


     |████████████████████████████████| 176 kB 7.9 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 


In [ ]:
pip install psaw


In [ ]:
import praw
import pandas as pd
import psaw
from psaw import PushshiftAPI
import datetime as dt
#initializing the PRAW client
prawRed = praw.Reddit(client_id="t6mBJIbJ2qcizwgRf-DAiQ",         # your client id
                               client_secret="3ooHwODsHcxKHfUN_WQpSLCHCUNlbw",      # your client secret
                               user_agent="Zubin Oommen")        # your user agent



api = PushshiftAPI(prawRed)

def convDate(year,month,date):
  return int(dt.datetime(year, month, date).timestamp())

end_epoch=convDate(2021, 12, 26)
start_epoch=convDate(2021, 12, 21)

# finds all the posts from a certain subreddit, within a certain timeframe
# this is the most time consuming part of the whole process
def getListPosts(subreddit, startDate, endDate):
    x=list(api.search_submissions(after=startDate,
                            before=endDate,
                            subreddit=subreddit,
                            filter=['url','author', 'title', 'subreddit', 'selftext'],
                            ))
    return x

data = getListPosts("Seahawks", start_epoch, end_epoch)




It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
#takes a single reddit post object, and turns it into an array. The array has the comments on the post, as well as the contents of the post itself
def createPostVec(submission, teamName, opponentName, winner):
  final=[]
  title=submission.title
  postText=submission.selftext
  upvotes=submission.score
  isComment=False
  date=submission.created_utc
  mainSub=[title, postText, upvotes, isComment,date, teamName, opponentName, winner]
  final.append(mainSub)
  for i in submission.comments:
    cT=i.body
    upvotes=i.score
    isComment=True
    cDate=i.created_utc
    commentArr=[title,cT,upvotes,isComment,cDate, teamName, opponentName, winner]
    final.append(commentArr)
  return final

##after some thought, decided i would not store comment threads. They become too chaotic and often resort to just one word posts which aren't useful.

print(createPostVec(data[4]))


In [ ]:
## Get schedules so you know what dates to output.
## should output an array with [team, opponent, date]
import bs4
from bs4 import BeautifulSoup
import requests
import datetime as dt

# Passes in a url and outputs a bs4 object
def getSoup(link):
    r=requests.get(link)
    soup=BeautifulSoup(r.text, 'html.parser')
    return soup

# passes in a team and year, makes the url, and outputs a bs4 object
def getTeamSoup(teamCode, season):
  url=r"https://www.espn.com/nfl/team/schedule/_/name/"+teamCode+r"/season/"+str(season)
  return getSoup(url)


# finds the index of the ESPN table in which the regular season starts
def getRegSeason(table):
  for i in range(0,len(table)):
    if "Regular Season" in table[i].text:
      return i
  return None

def getPostSeason(table):
  for i in range(0,len(table)):
    if "Postseason" in table[i].text:
      return i
  return None


# finds the index of the ESPN table in which the regular season ends (preseason starts)
def getPreSeason(table):
  for i in range(0,len(table)):
    if "Preseason" in table[i].text:
      return i
  return None

# uses the two indices found above to create the bs4 sections we need for our model.
def getSznData(teamCode, season):
  soup=getTeamSoup(teamCode, season)
  table=soup.find_all('tr')
  startPoint=getRegSeason(table)+2
  endPoint=getPreSeason(table)
  return(table[startPoint:endPoint])

def getPostData(teamCode, season):
  soup=getTeamSoup(teamCode, season)
  table=soup.find_all('tr')
  startPoint=getPostSeason(table)+2
  endPoint=getRegSeason(table)
  return(table[startPoint:endPoint])

# takes the bs4 sections found above, converts them into usable text and formats in an array. 
def makeDataArr(teamCode, season):
  usefulData=getSznData(teamCode, season)
  finalArr=[]
  for i in usefulData:
    temp=i.find_all('td')
    try:
      date=dateToUTC(temp[1].text.strip(),season)
      opponent=temp[2].text[2:].strip()
      result=temp[3].text[:1].strip()
      if opponent=="Los Angeles":
        pass
      elif opponent=="New York":
        pass
      else:
        tempArr=[teamCode,date,opponent,result]
        finalArr.append(tempArr)
    except:
      # should only get here if it is a "BYE WEEK" = no data
      pass
  return finalArr


def makePostDataArr(teamCode, season):
  usefulData=getPostData(teamCode, season)
  finalArr=[]
  for i in usefulData:
    temp=i.find_all('td')
    try:
      date=dateToUTC(temp[1].text.strip(),season)
      opponent=temp[2].text[2:].strip()
      result=temp[3].text[:1].strip()
      tempArr=[teamCode,date,opponent,result]
      finalArr.append(tempArr)
    except:
      # should only get here if it is a "BYE WEEK" = no data
      pass
  return finalArr

## Converts numeric date values to a UTC
def convDate(year,month,date):
  return int(dt.datetime(year, month, date).timestamp())

monthDict = {
  "Aug": 8,
  "Sep": 9,
  "Oct": 10,
  "Nov": 11,
  "Dec": 12,
  "Jan": 1,
  "Feb": 2
}

teamDict = {
    "ari":"Arizona",
    "sea":"Seattle",
    "sf":"San Francisco",
    "lar":"Los Angeles",
    "gb":"Green Bay",
    "min":"Minnesota",
    "chi":"Chicago",
    "det":"Detroit",
    "dal":"Dallas",
    "wsh":"Washington",
 

}


## Converts a date from the ESPN scraped format into a usable UTC value for PSAW.
def dateToUTC(date, year):
  arr=date.split()
  month=arr[1]
  date=int(arr[2])
  monthNum=monthDict[month]
  if monthNum>7:
    fYear=year
  else:
    fYear=year+1
  return(convDate(fYear,monthNum,date))

makeDataArr("ind", 2021)
makePostDataArr("lar",2021)

[['lar', 1642377600, 'Arizona', 'W'],
 ['lar', 1642896000, 'Tampa Bay', 'W'],
 ['lar', 1643500800, 'San Francisco', 'W'],
 ['lar', 1644710400, 'Cincinnati *', 'W']]

In [ ]:
import csv
redditDict = {
    "Seattle": "seahawks",
    "Arizona": "AZCardinals",
    "Los Angeles": "LosAngelesRams",
    "San Francisco": "49ers",
    "Green Bay": "GreenBayPackers",
    "Minnesota": "minnesotavikings",
    "Detroit": "detroitlions",
    "Chicago": "CHIBears",
    "Dallas": "cowboys",
    "Washington": "Commanders",
    #"New York": "NYGiants"
    "Philadelphia":"eagles",
    "Tampa Bay": "buccaneers",
    "Carolina": "panthers",
    "New Orleans": "saints",
    "Atlanta":"falcons",
    #"Los Angeles": "chargers"
    "Kansas City": "KansasCityChiefs",
    "Denver": "DenverBroncos",
    "Las Vegas": "raiders",
    "Pittsburgh": "steelers",
    "Cincinnati": "bengals",
    "Baltimore": "ravens",
    "Cleveland": "browns",
    "Jacksonville": "jaguars",
    "Tennessee": "Tennesseetitans",
    "Houston": "texans",
    "Indianapolis": "Colts",
    #"New York": "nyjets",
    "New England": "Patriots",
    "Buffalo": "buffalobills",
    "Miami": "miamidolphins"
}

codeDict={
    "sea":"Seattle",
    "ari":"Arizona",
    "sf": "San Francisco",
    "gb": "Green Bay",
    "min": "Minnesota",
    "det": "Detroit",
    "chi":"Chicago",
    "dal":"Dallas",
    "wsh": "Washington",
    "phi": "Philadelphia",
    "tb":"Tampa Bay",
    "car":"Carolina",
    "no": "New Orleans",
    "atl":"Atlanta",
    "kc":"Kansas City",
    "den":"Denver",
    "lv":"Las Vegas",
    "pit":"Pittsburgh",
    "cin":"Cincinnati",
    "bal":"Baltimore",
    "cle":"Cleveland",
    "jax":"Jacksonville",
    "ten":"Tennessee",
    "hou":"Houston",
    "ind":"Indianapolis",
    "ne":"New England",
    "buf":"Buffalo",
    "mia":"Miami"


}

file_path=r"C:\Users\zoommen\Documents\Junior Year\Spring Tri\Independent Study\reddit_data.csv"

def writeToCSV(arr):
  file = open(file_path, 'w+', newline ='')

  with file:
    write=csv.writer(file)
    write.writerows(test)

def matchData(arrToMatch):
  teamLoc=codeDict[arrToMatch[0]]
  TeamSubReddit=redditDict[teamLoc]
  OppSubReddit=redditDict[arrToMatch[2]]
  if arrToMatch[3]=="W":
    winner=TeamSubReddit
  else:
    winner=OppSubReddit
  startDate=arrToMatch[1]-518400 #6 days before the game
  endDate=arrToMatch[1]-129600 #1 day before the game
  data=getListPosts(TeamSubReddit, startDate, endDate)
  print(data)
  #fullData=[]
  for i in data:
    text=createPostVec(i, arrToMatch[0], arrToMatch[2], winner)
    print(text)
    writeToCSV(text)
    #fullData.append(createPostVec(i, arrToMatch[0], arrToMatch[2], winner))

#matchData( ['Seattle', 1609632000, 'San Francisco', 'W'])

In [ ]:
ALL_DATA_2021=[['sea', 1631404800, 'Indianapolis', 'W'], ['sea', 1632009600, 'Tennessee', 'L'], ['sea', 1632614400, 'Minnesota', 'L'], ['sea', 1633219200, 'San Francisco', 'W'], ['sea', 1634428800, 'Pittsburgh', 'L'], ['sea', 1635120000, 'New Orleans', 'L'], ['sea', 1635638400, 'Jacksonville', 'W'], ['sea', 1636848000, 'Green Bay', 'L'], ['sea', 1637452800, 'Arizona', 'L'], ['sea', 1638144000, 'Washington', 'L'], ['sea', 1638662400, 'San Francisco', 'W'], ['sea', 1639267200, 'Houston', 'W'], ['sea', 1640476800, 'Chicago', 'L'], ['sea', 1641081600, 'Detroit', 'W'], ['sea', 1641686400, 'Arizona', 'W'], ['ari', 1631404800, 'Tennessee', 'W'], ['ari', 1632009600, 'Minnesota', 'W'], ['ari', 1632614400, 'Jacksonville', 'W'], ['ari', 1633824000, 'San Francisco', 'W'], ['ari', 1634428800, 'Cleveland', 'W'], ['ari', 1635033600, 'Houston', 'W'], ['ari', 1635379200, 'Green Bay', 'L'], ['ari', 1636243200, 'San Francisco', 'W'], ['ari', 1636848000, 'Carolina', 'L'], ['ari', 1637452800, 'Seattle', 'W'], ['ari', 1638662400, 'Chicago', 'W'], ['ari', 1639872000, 'Detroit', 'L'], ['ari', 1640390400, 'Indianapolis', 'L'], ['ari', 1641081600, 'Dallas', 'W'], ['ari', 1641686400, 'Seattle', 'L'], ['sf', 1631404800, 'Detroit', 'W'], ['sf', 1632009600, 'Philadelphia', 'W'], ['sf', 1632614400, 'Green Bay', 'L'], ['sf', 1633219200, 'Seattle', 'L'], ['sf', 1633824000, 'Arizona', 'L'], ['sf', 1635033600, 'Indianapolis', 'L'], ['sf', 1635638400, 'Chicago', 'W'], ['sf', 1636243200, 'Arizona', 'L'], ['sf', 1637452800, 'Jacksonville', 'W'], ['sf', 1638057600, 'Minnesota', 'W'], ['sf', 1638662400, 'Seattle', 'L'], ['sf', 1639267200, 'Cincinnati', 'W'], ['sf', 1639872000, 'Atlanta', 'W'], ['sf', 1640217600, 'Tennessee', 'L'], ['sf', 1641081600, 'Houston', 'W'], ['gb', 1631404800, 'New Orleans', 'L'], ['gb', 1632096000, 'Detroit', 'W'], ['gb', 1632614400, 'San Francisco', 'W'], ['gb', 1633219200, 'Pittsburgh', 'W'], ['gb', 1633824000, 'Cincinnati', 'W'], ['gb', 1634428800, 'Chicago', 'W'], ['gb', 1635033600, 'Washington', 'W'], ['gb', 1635379200, 'Arizona', 'W'], ['gb', 1636243200, 'Kansas City', 'L'], ['gb', 1636848000, 'Seattle', 'W'], ['gb', 1637452800, 'Minnesota', 'L'], ['gb', 1639267200, 'Chicago', 'W'], ['gb', 1639872000, 'Baltimore', 'W'], ['gb', 1640390400, 'Cleveland', 'W'], ['gb', 1641081600, 'Minnesota', 'W'], ['gb', 1641686400, 'Detroit', 'L'], ['min', 1631404800, 'Cincinnati', 'L'], ['min', 1632009600, 'Arizona', 'L'], ['min', 1632614400, 'Seattle', 'W'], ['min', 1633219200, 'Cleveland', 'L'], ['min', 1633824000, 'Detroit', 'W'], ['min', 1634428800, 'Carolina', 'W'], ['min', 1635638400, 'Dallas', 'L'], ['min', 1636243200, 'Baltimore', 'L'], ['min', 1637452800, 'Green Bay', 'W'], ['min', 1638057600, 'San Francisco', 'L'], ['min', 1638662400, 'Detroit', 'L'], ['min', 1639008000, 'Pittsburgh', 'W'], ['min', 1639958400, 'Chicago', 'W'], ['min', 1641081600, 'Green Bay', 'L'], ['min', 1641686400, 'Chicago', 'W'], ['det', 1631404800, 'San Francisco', 'L'], ['det', 1632096000, 'Green Bay', 'L'], ['det', 1632614400, 'Baltimore', 'L'], ['det', 1633219200, 'Chicago', 'L'], ['det', 1633824000, 'Minnesota', 'L'], ['det', 1634428800, 'Cincinnati', 'L'], ['det', 1635638400, 'Philadelphia', 'L'], ['det', 1636848000, 'Pittsburgh', 'T'], ['det', 1637452800, 'Cleveland', 'L'], ['det', 1637798400, 'Chicago', 'L'], ['det', 1638662400, 'Minnesota', 'W'], ['det', 1639267200, 'Denver', 'L'], ['det', 1639872000, 'Arizona', 'W'], ['det', 1640476800, 'Atlanta', 'L'], ['det', 1641081600, 'Seattle', 'L'], ['det', 1641686400, 'Green Bay', 'W'], ['chi', 1632009600, 'Cincinnati', 'W'], ['chi', 1632614400, 'Cleveland', 'L'], ['chi', 1633219200, 'Detroit', 'W'], ['chi', 1633824000, 'Las Vegas', 'W'], ['chi', 1634428800, 'Green Bay', 'L'], ['chi', 1635033600, 'Tampa Bay', 'L'], ['chi', 1635638400, 'San Francisco', 'L'], ['chi', 1636329600, 'Pittsburgh', 'L'], ['chi', 1637452800, 'Baltimore', 'L'], ['chi', 1637798400, 'Detroit', 'W'], ['chi', 1638662400, 'Arizona', 'L'], ['chi', 1639267200, 'Green Bay', 'L'], ['chi', 1639958400, 'Minnesota', 'L'], ['chi', 1640476800, 'Seattle', 'W'], ['chi', 1641686400, 'Minnesota', 'L'], ['phi', 1631404800, 'Atlanta', 'W'], ['phi', 1632009600, 'San Francisco', 'L'], ['phi', 1632700800, 'Dallas', 'L'], ['phi', 1633219200, 'Kansas City', 'L'], ['phi', 1633824000, 'Carolina', 'W'], ['phi', 1634169600, 'Tampa Bay', 'L'], ['phi', 1635033600, 'Las Vegas', 'L'], ['phi', 1635638400, 'Detroit', 'W'], ['phi', 1636848000, 'Denver', 'W'], ['phi', 1637452800, 'New Orleans', 'W'], ['phi', 1640044800, 'Washington', 'W'], ['phi', 1641081600, 'Washington', 'W'], ['phi', 1641600000, 'Dallas', 'L'], ['wsh', 1632614400, 'Buffalo', 'L'], ['wsh', 1633219200, 'Atlanta', 'W'], ['wsh', 1633824000, 'New Orleans', 'L'], ['wsh', 1634428800, 'Kansas City', 'L'], ['wsh', 1635033600, 'Green Bay', 'L'], ['wsh', 1635638400, 'Denver', 'L'], ['wsh', 1636848000, 'Tampa Bay', 'W'], ['wsh', 1637452800, 'Carolina', 'W'], ['wsh', 1638144000, 'Seattle', 'W'], ['wsh', 1638662400, 'Las Vegas', 'W'], ['wsh', 1639267200, 'Dallas', 'L'], ['wsh', 1640044800, 'Philadelphia', 'L'], ['wsh', 1640476800, 'Dallas', 'L'], ['wsh', 1641081600, 'Philadelphia', 'L'], ['dal', 1631145600, 'Tampa Bay', 'L'], ['dal', 1632700800, 'Philadelphia', 'W'], ['dal', 1633219200, 'Carolina', 'W'], ['dal', 1634428800, 'New England', 'W'], ['dal', 1635638400, 'Minnesota', 'W'], ['dal', 1636243200, 'Denver', 'L'], ['dal', 1636848000, 'Atlanta', 'W'], ['dal', 1637452800, 'Kansas City', 'L'], ['dal', 1637798400, 'Las Vegas', 'L'], ['dal', 1638403200, 'New Orleans', 'W'], ['dal', 1639267200, 'Washington', 'W'], ['dal', 1640476800, 'Washington', 'W'], ['dal', 1641081600, 'Arizona', 'L'], ['dal', 1641600000, 'Philadelphia', 'W'], ['tb', 1631145600, 'Dallas', 'W'], ['tb', 1632009600, 'Atlanta', 'W'], ['tb', 1633219200, 'New England', 'W'], ['tb', 1633824000, 'Miami', 'W'], ['tb', 1634169600, 'Philadelphia', 'W'], ['tb', 1635033600, 'Chicago', 'W'], ['tb', 1635638400, 'New Orleans', 'L'], ['tb', 1636848000, 'Washington', 'L'], ['tb', 1638057600, 'Indianapolis', 'W'], ['tb', 1638662400, 'Atlanta', 'W'], ['tb', 1639267200, 'Buffalo', 'W'], ['tb', 1639872000, 'New Orleans', 'L'], ['tb', 1640476800, 'Carolina', 'W'], ['tb', 1641686400, 'Carolina', 'W'], ['car', 1632009600, 'New Orleans', 'W'], ['car', 1632355200, 'Houston', 'W'], ['car', 1633219200, 'Dallas', 'L'], ['car', 1633824000, 'Philadelphia', 'L'], ['car', 1634428800, 'Minnesota', 'L'], ['car', 1635638400, 'Atlanta', 'W'], ['car', 1636243200, 'New England', 'L'], ['car', 1636848000, 'Arizona', 'W'], ['car', 1637452800, 'Washington', 'L'], ['car', 1638057600, 'Miami', 'L'], ['car', 1639267200, 'Atlanta', 'L'], ['car', 1639872000, 'Buffalo', 'L'], ['car', 1640476800, 'Tampa Bay', 'L'], ['car', 1641081600, 'New Orleans', 'L'], ['car', 1641686400, 'Tampa Bay', 'L'], ['atl', 1631404800, 'Philadelphia', 'L'], ['atl', 1632009600, 'Tampa Bay', 'L'], ['atl', 1633219200, 'Washington', 'L'], ['atl', 1633824000, 'New York *', 'W'], ['atl', 1635033600, 'Miami', 'W'], ['atl', 1635638400, 'Carolina', 'L'], ['atl', 1636243200, 'New Orleans', 'W'], ['atl', 1636848000, 'Dallas', 'L'], ['atl', 1637193600, 'New England', 'L'], ['atl', 1638057600, 'Jacksonville', 'W'], ['atl', 1638662400, 'Tampa Bay', 'L'], ['atl', 1639267200, 'Carolina', 'W'], ['atl', 1639872000, 'San Francisco', 'L'], ['atl', 1640476800, 'Detroit', 'W'], ['atl', 1641081600, 'Buffalo', 'L'], ['atl', 1641686400, 'New Orleans', 'L'], ['no', 1631404800, 'Green Bay', 'W'], ['no', 1632009600, 'Carolina', 'L'], ['no', 1632614400, 'New England', 'W'], ['no', 1633824000, 'Washington', 'W'], ['no', 1635120000, 'Seattle', 'W'], ['no', 1635638400, 'Tampa Bay', 'W'], ['no', 1636243200, 'Atlanta', 'L'], ['no', 1636848000, 'Tennessee', 'L'], ['no', 1637452800, 'Philadelphia', 'L'], ['no', 1637798400, 'Buffalo', 'L'], ['no', 1638403200, 'Dallas', 'L'], ['no', 1639872000, 'Tampa Bay', 'W'], ['no', 1640563200, 'Miami', 'L'], ['no', 1641081600, 'Carolina', 'W'], ['no', 1641686400, 'Atlanta', 'W'], ['kc', 1631404800, 'Cleveland', 'W'], ['kc', 1632009600, 'Baltimore', 'L'], ['kc', 1633219200, 'Philadelphia', 'W'], ['kc', 1633824000, 'Buffalo', 'L'], ['kc', 1634428800, 'Washington', 'W'], ['kc', 1635033600, 'Tennessee', 'L'], ['kc', 1636243200, 'Green Bay', 'W'], ['kc', 1636848000, 'Las Vegas', 'W'], ['kc', 1637452800, 'Dallas', 'W'], ['kc', 1638662400, 'Denver', 'W'], ['kc', 1639267200, 'Las Vegas', 'W'], ['kc', 1640476800, 'Pittsburgh', 'W'], ['kc', 1641081600, 'Cincinnati', 'L'], ['kc', 1641600000, 'Denver', 'W'], ['cin', 1631404800, 'Minnesota', 'W'], ['cin', 1632009600, 'Chicago', 'L'], ['cin', 1632614400, 'Pittsburgh', 'W'], ['cin', 1632960000, 'Jacksonville', 'W'], ['cin', 1633824000, 'Green Bay', 'L'], ['cin', 1634428800, 'Detroit', 'W'], ['cin', 1635033600, 'Baltimore', 'W'], ['cin', 1636243200, 'Cleveland', 'L'], ['cin', 1637452800, 'Las Vegas', 'W'], ['cin', 1638057600, 'Pittsburgh', 'W'], ['cin', 1639267200, 'San Francisco', 'L'], ['cin', 1639872000, 'Denver', 'W'], ['cin', 1640476800, 'Baltimore', 'W'], ['cin', 1641081600, 'Kansas City', 'W'], ['cin', 1641686400, 'Cleveland', 'L'], ['lv', 1631491200, 'Baltimore', 'W'], ['lv', 1632009600, 'Pittsburgh', 'W'], ['lv', 1632614400, 'Miami', 'W'], ['lv', 1633824000, 'Chicago', 'L'], ['lv', 1634428800, 'Denver', 'W'], ['lv', 1635033600, 'Philadelphia', 'W'], ['lv', 1636848000, 'Kansas City', 'L'], ['lv', 1637452800, 'Cincinnati', 'L'], ['lv', 1637798400, 'Dallas', 'W'], ['lv', 1638662400, 'Washington', 'L'], ['lv', 1639267200, 'Kansas City', 'L'], ['lv', 1639958400, 'Cleveland', 'W'], ['lv', 1640476800, 'Denver', 'W'], ['lv', 1641081600, 'Indianapolis', 'W'], ['den', 1632009600, 'Jacksonville', 'W'], ['den', 1633219200, 'Baltimore', 'L'], ['den', 1633824000, 'Pittsburgh', 'L'], ['den', 1634428800, 'Las Vegas', 'L'], ['den', 1634774400, 'Cleveland', 'L'], ['den', 1635638400, 'Washington', 'W'], ['den', 1636243200, 'Dallas', 'W'], ['den', 1636848000, 'Philadelphia', 'L'], ['den', 1638662400, 'Kansas City', 'L'], ['den', 1639267200, 'Detroit', 'W'], ['den', 1639872000, 'Cincinnati', 'L'], ['den', 1640476800, 'Las Vegas', 'L'], ['den', 1641600000, 'Kansas City', 'L'], ['pit', 1631404800, 'Buffalo', 'W'], ['pit', 1632009600, 'Las Vegas', 'L'], ['pit', 1632614400, 'Cincinnati', 'L'], ['pit', 1633219200, 'Green Bay', 'L'], ['pit', 1633824000, 'Denver', 'W'], ['pit', 1634428800, 'Seattle', 'W'], ['pit', 1635638400, 'Cleveland', 'W'], ['pit', 1636329600, 'Chicago', 'W'], ['pit', 1636848000, 'Detroit', 'T'], ['pit', 1638057600, 'Cincinnati', 'L'], ['pit', 1638662400, 'Baltimore', 'W'], ['pit', 1639008000, 'Minnesota', 'L'], ['pit', 1639872000, 'Tennessee', 'W'], ['pit', 1640476800, 'Kansas City', 'L'], ['pit', 1641168000, 'Cleveland', 'W'], ['pit', 1641686400, 'Baltimore', 'W'], ['cle', 1631404800, 'Kansas City', 'L'], ['cle', 1632009600, 'Houston', 'W'], ['cle', 1632614400, 'Chicago', 'W'], ['cle', 1633219200, 'Minnesota', 'W'], ['cle', 1634428800, 'Arizona', 'L'], ['cle', 1634774400, 'Denver', 'W'], ['cle', 1635638400, 'Pittsburgh', 'L'], ['cle', 1636243200, 'Cincinnati', 'W'], ['cle', 1636848000, 'New England', 'L'], ['cle', 1637452800, 'Detroit', 'W'], ['cle', 1638057600, 'Baltimore', 'L'], ['cle', 1639267200, 'Baltimore', 'W'], ['cle', 1639958400, 'Las Vegas', 'L'], ['cle', 1640390400, 'Green Bay', 'L'], ['cle', 1641168000, 'Pittsburgh', 'L'], ['cle', 1641686400, 'Cincinnati', 'W'], ['bal', 1631491200, 'Las Vegas', 'L'], ['bal', 1632009600, 'Kansas City', 'W'], ['bal', 1632614400, 'Detroit', 'W'], ['bal', 1633219200, 'Denver', 'W'], ['bal', 1633910400, 'Indianapolis', 'W'], ['bal', 1635033600, 'Cincinnati', 'L'], ['bal', 1636243200, 'Minnesota', 'W'], ['bal', 1636588800, 'Miami', 'L'], ['bal', 1637452800, 'Chicago', 'W'], ['bal', 1638057600, 'Cleveland', 'W'], ['bal', 1638662400, 'Pittsburgh', 'L'], ['bal', 1639267200, 'Cleveland', 'L'], ['bal', 1639872000, 'Green Bay', 'L'], ['bal', 1640476800, 'Cincinnati', 'L'], ['bal', 1641686400, 'Pittsburgh', 'L'], ['ne', 1631404800, 'Miami', 'L'], ['ne', 1632614400, 'New Orleans', 'L'], ['ne', 1633219200, 'Tampa Bay', 'L'], ['ne', 1633824000, 'Houston', 'W'], ['ne', 1634428800, 'Dallas', 'L'], ['ne', 1636243200, 'Carolina', 'W'], ['ne', 1636848000, 'Cleveland', 'W'], ['ne', 1637193600, 'Atlanta', 'W'], ['ne', 1638057600, 'Tennessee', 'W'], ['ne', 1638748800, 'Buffalo', 'W'], ['ne', 1639785600, 'Indianapolis', 'L'], ['ne', 1640476800, 'Buffalo', 'L'], ['ne', 1641081600, 'Jacksonville', 'W'], ['ne', 1641686400, 'Miami', 'L'], ['mia', 1631404800, 'New England', 'W'], ['mia', 1632009600, 'Buffalo', 'L'], ['mia', 1632614400, 'Las Vegas', 'L'], ['mia', 1633219200, 'Indianapolis', 'L'], ['mia', 1633824000, 'Tampa Bay', 'L'], ['mia', 1634428800, 'Jacksonville *', 'L'], ['mia', 1635033600, 'Atlanta', 'L'], ['mia', 1635638400, 'Buffalo', 'L'], ['mia', 1636243200, 'Houston', 'W'], ['mia', 1636588800, 'Baltimore', 'W'], ['mia', 1638057600, 'Carolina', 'W'], ['mia', 1640563200, 'New Orleans', 'W'], ['mia', 1641081600, 'Tennessee', 'L'], ['mia', 1641686400, 'New England', 'W'], ['buf', 1631404800, 'Pittsburgh', 'L'], ['buf', 1632009600, 'Miami', 'W'], ['buf', 1632614400, 'Washington', 'W'], ['buf', 1633219200, 'Houston', 'W'], ['buf', 1633824000, 'Kansas City', 'W'], ['buf', 1634515200, 'Tennessee', 'L'], ['buf', 1635638400, 'Miami', 'W'], ['buf', 1636243200, 'Jacksonville', 'L'], ['buf', 1637452800, 'Indianapolis', 'L'], ['buf', 1637798400, 'New Orleans', 'W'], ['buf', 1638748800, 'New England', 'L'], ['buf', 1639267200, 'Tampa Bay', 'L'], ['buf', 1639872000, 'Carolina', 'W'], ['buf', 1640476800, 'New England', 'W'], ['buf', 1641081600, 'Atlanta', 'W'], ['hou', 1631404800, 'Jacksonville', 'W'], ['hou', 1632009600, 'Cleveland', 'L'], ['hou', 1632355200, 'Carolina', 'L'], ['hou', 1633219200, 'Buffalo', 'L'], ['hou', 1633824000, 'New England', 'L'], ['hou', 1634428800, 'Indianapolis', 'L'], ['hou', 1635033600, 'Arizona', 'L'], ['hou', 1636243200, 'Miami', 'L'], ['hou', 1637452800, 'Tennessee', 'W'], ['hou', 1638662400, 'Indianapolis', 'L'], ['hou', 1639267200, 'Seattle', 'L'], ['hou', 1639872000, 'Jacksonville', 'W'], ['hou', 1641081600, 'San Francisco', 'L'], ['hou', 1641686400, 'Tennessee', 'L'],['jax', 1631404800, 'Houston', 'L'], ['jax', 1632009600, 'Denver', 'L'], ['jax', 1632614400, 'Arizona', 'L'], ['jax', 1632960000, 'Cincinnati', 'L'], ['jax', 1633824000, 'Tennessee', 'L'], ['jax', 1634428800, 'Miami *', 'W'], ['jax', 1635638400, 'Seattle', 'L'], ['jax', 1636243200, 'Buffalo', 'W'], ['jax', 1636848000, 'Indianapolis', 'L'], ['jax', 1637452800, 'San Francisco', 'L'], ['jax', 1638057600, 'Atlanta', 'L'], ['jax', 1639267200, 'Tennessee', 'L'], ['jax', 1639872000, 'Houston', 'L'], ['jax', 1641081600, 'New England', 'L'], ['jax', 1641686400, 'Indianapolis', 'W'], ['ten', 1631404800, 'Arizona', 'L'], ['ten', 1632009600, 'Seattle', 'W'], ['ten', 1632614400, 'Indianapolis', 'W'], ['ten', 1633824000, 'Jacksonville', 'W'], ['ten', 1634515200, 'Buffalo', 'W'], ['ten', 1635033600, 'Kansas City', 'W'], ['ten', 1635638400, 'Indianapolis', 'W'], ['ten', 1636848000, 'New Orleans', 'W'], ['ten', 1637452800, 'Houston', 'L'], ['ten', 1638057600, 'New England', 'L'], ['ten', 1639267200, 'Jacksonville', 'W'], ['ten', 1639872000, 'Pittsburgh', 'L'], ['ten', 1640217600, 'San Francisco', 'W'], ['ten', 1641081600, 'Miami', 'W'], ['ten', 1641686400, 'Houston', 'W'], ['ind', 1631404800, 'Seattle', 'L'], ['ind', 1632614400, 'Tennessee', 'L'], ['ind', 1633219200, 'Miami', 'W'], ['ind', 1633910400, 'Baltimore', 'L'], ['ind', 1634428800, 'Houston', 'W'], ['ind', 1635033600, 'San Francisco', 'W'], ['ind', 1635638400, 'Tennessee', 'L'], ['ind', 1636848000, 'Jacksonville', 'W'], ['ind', 1637452800, 'Buffalo', 'W'], ['ind', 1638057600, 'Tampa Bay', 'L'], ['ind', 1638662400, 'Houston', 'W'], ['ind', 1639785600, 'New England', 'W'], ['ind', 1640390400, 'Arizona', 'W'], ['ind', 1641081600, 'Las Vegas', 'L'], ['ind', 1641686400, 'Jacksonville', 'L']]

In [ ]:
for i in ALL_DATA_2021:
    matchData(i)

413